In [1]:
from scipy.constants import epsilon_0
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline
from SimPEG.Utils import ndgrid
from ipywidgets import *

# Setup:

Sphere in a wholespace with a constant, uniform electric field

Parameters:

 - sigma_0 : conductivity of the background
 - sigma_1 : conductivity of the sphere

# Make a plot 

x = np.linspace(0,1,50)
m = 1
b = 10
y = m*x + b

fig,ax = plt.subplots(1,2,figsize=(4,5))
ax[0].plot(x,y)

ax[1].plot(y,x)

#Integer/Float

print 1/3   # integer division
print 1./3. # floats 

# Parameters for the sphere Problem


In [9]:

sig0 = 1        # conductivity of the wholespace
sig1 = 2           # conductivity of the sphere
R    = 50          # radius of the sphere
E0   = 1           # inducing field strength
xr = np.linspace(-2.*R, 2.*R, 100)
yr = xr.copy()
zr = np.r_[0] # identical to saying `zr = np.array([0])`
XYZ = ndgrid(xr,yr,zr) #Half-Space Definition
# d    = 2           # mesh spacing

In [10]:
r  = lambda x,y,z: np.sqrt(x**2+y**2+z**2)

In [63]:
# assumes an x-oriented inducing field
# and that the sphere is at the origin
def get_Potential(XYZ,sig0,sig1,R,E0): 
    
    x,y,z = XYZ[:,0],XYZ[:,1],XYZ[:,2]
    
    sigf = (sig1-sig0)/(sig1+2.*sig0)
    
    r_cur = r(x,y,z)
    
    ind0 = (r_cur > R)
    ind1 = (r_cur <= R)
    
    assert (ind0 + ind1).all(), 'Some indicies not included'
    
    V = np.zeros_like(x)
    
    V[ind0] = -E0*x[ind0]*(1.-sigf*R**3./r_cur[ind0]**3) # outside the sphere
    V[ind1] = -E0*x[ind1]*3.*sig0/(sig1+2.*sig0)
    
    return V

In [64]:
def get_Primary_Potential(XYZ,sig0,sig1,R,E0):
    x = XYZ[:,0]
    return - E0*x

In [65]:
XYZ.shape

(10000, 3)

In [74]:
def get_Conductivity(XYZ,sig0,sig1,R):
    x,y,z = XYZ[:,0],XYZ[:,1],XYZ[:,2]
    r_view=r(x,y,z)
    
    int0= (r_view>R)
    int1= (r_view<=R)
    
    Sigma = np.zeros_like(x)
    
    Sigma[int0]=sig0
    Sigma[int1]=sig1
    
    return Sigma

In [ ]:
R_slider = FloatSlider(min=0., max =50., step=10.)
S_slider = FloatSlider(min=-10., max =10., step=1.)

def plot_Potential(R,sig1):
    Sigma = get_Conductivity(XYZ,sig0,sig1,R)
    V = get_Potential(XYZ,sig0,sig1,R,E0)
    Vp = get_Primary_Potential(XYZ,sig0,sig1,R,E0)

    xcirc = xr[np.abs(xr) <= R]

    fig,ax = plt.subplots(1,4,figsize=(10,5))

    ax[0].pcolor(xr,yr,Sigma.reshape(xr.size, yr.size))
    ax[0].plot(xcirc,np.sqrt(R**2-xcirc**2),'--k',xcirc,-np.sqrt(R**2-xcirc**2),'--k')
    ax[0].set_title('Configuration')

    ax[1].pcolor(xr,yr,Vp.reshape(xr.size,yr.size))
    ax[1].plot(xcirc,np.sqrt(R**2-xcirc**2),'--k',xcirc,-np.sqrt(R**2-xcirc**2),'--k')
    ax[1].set_title('Primary Potential')

    ax[2].pcolor(xr,yr,V.reshape(xr.size,yr.size))
    ax[2].plot(xcirc,np.sqrt(R**2-xcirc**2),'--k',xcirc,-np.sqrt(R**2-xcirc**2),'--k')
    ax[2].set_title('Total Potential')

    ax[3].pcolor(xr,yr,(V-Vp).reshape(xr.size,yr.size))
    ax[3].plot(xcirc,np.sqrt(R**2-xcirc**2),'--k',xcirc,-np.sqrt(R**2-xcirc**2),'--k')
    ax[3].set_title('Secondary Potential')

    plt.tight_layout()
    
interact(plot_Problem, R=R_slider, sig1=S_slider)

In [88]:
R_slider = FloatSlider(min=0., max =50., step=10.)
S_slider = FloatSlider(min=-10., max =10., step=1.)

Interact(R_slider, S_slider)

NameError: name 'Interact' is not defined

E1x = @(x,y,z,sig1,sig2) E0 + E0*R^3./r(x,y,z).^5.*sigf(sig1,sig2).*(2*x.^2-y.^2-z.^2);
E1y = @(x,y,z,sig1,sig2) E0*R^3./r(x,y,z).^5.*3.*x.*y*sigf(sig1,sig2);
E1z = @(x,y,z,sig1,sig2) E0*R^3./r(x,y,z).^5.*3.*x.*z*sigf(sig1,sig2);

E2x = @(x,y,z,sig1,sig2) 3*sig1/(sig2+2*sig1)*ones(size(x));
E2y = @(x,y,z,sig1,sig2) 0*ones(size(x));
E2z = @(x,y,z,sig1,sig2) 0*ones(size(x));

Ex = @(x,y,z,sig1,sig2) (r(x,y,z) > R).*E1x(x,y,z,sig1,sig2) + (r(x,y,z) <= R).*E2x(x,y,z,sig1,sig2);
Ey = @(x,y,z,sig1,sig2) (r(x,y,z) > R).*E1y(x,y,z,sig1,sig2) + (r(x,y,z) <= R).*E2y(x,y,z,sig1,sig2); 
Ez = @(x,y,z,sig1,sig2) (r(x,y,z) > R).*E1z(x,y,z,sig1,sig2) + (r(x,y,z) <= R).*E2z(x,y,z,sig1,sig2);

Exp = @(x,y,z,sig1,sig2) E0.*ones(size(x));